<a href="https://colab.research.google.com/github/rociohbarroso/Python/blob/main/Prueba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#pip install tensorflow
#pip install keras
#pip install imgaug
#pip install opencv-python
#pip install h5py
#pip install tqdm
#pip install imutils

import argparse
import numpy as np
import json
import cv2
import copy
import imgaug as ia
from imgaug import augmenters as iaa
from keras.utils import Sequence
import xml.etree.ElementTree as ET


import os
imagenes_path = "/imagenes"
anotaciones_path="/anotaciones"

trainval = open(os.path.join(anotaciones_path,"documento.xml")).readlines()
os.makedirs(os.path.join(imagenes_path,"train","lubinas"), exist_ok=True)

def classify_image(line, subset):
  basename = line.split(" ")[0]
  species = line.split(" ")[2]
  oldpath = os.path.join(imagenes_path, f"{basename}.jpg")
  newpath = os.path.join(imagenes_path, subset, f"{basename}.jpg")
  if os.path.isfile(oldpath):
        os.rename(oldpath, newpath)


from tensorflow.keras.preprocessing.image import ImageDataGenerator

generador_entrenamiento = ImageDataGenerator()
datos_entrenamiento = generador_entrenamiento.flow_from_directory("/imagenes/")
generador_test = ImageDataGenerator()
datos_test = generador_test.flow_from_directory("/imagenes/", class_mode=None)
algunas_imagenes = next(datos_test)



from matplotlib import pyplot as plt
plt.imshow(algunas_imagenes[0]/255.)
plt.axis('off')
plt.show()



from tensorflow.keras import applications
inception = applications.InceptionV3(include_top=False, input_shape=(256, 256, 3))

from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
predictor = Sequential([
    Flatten(),
    Dense(128, activation="relu"),
    Dense(2, activation="softmax")
])
modelo = Sequential([inception, predictor])
modelo.compile(optimizer="adam", loss="categorical_crossentropy")

modelo.fit(datos_entrenamiento, epochs=1)

def bbox_iou(box1, box2):
  intersect_w = _interval_overlap([box1.xmin, box1.max], [box2.xmin, box2.max])
  intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.yax])

  intersect = intersect_w * intersect_h

  w1,h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
  w2,h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin

  union = w1*h1 + w2*h2 - intersect

  return float(intersect) / union

class BoundBox:
  def __init__(self, xmin, ymin, xmax, ymax, c = None, classes = None):
    self.xmin = xmin
    self.ymin = ymin
    self.xmax = xmax
    self.ymax = ymax
    self.c = c
    self.classes = classes
    self.label = -1
    self.score = -1

  def get_label(self):
    if self.label == -1:
      self.label = np.argmax(self.classes)
    return self.label

  def get_score(self):
    if self.score == -1:
      self.core = self.classes[self.get_label()]
    return self.score

class BatchGenerator(Sequence):
  def __init__(self, images, config, shuffle=True, jitter=True, norm=None):
    self.generator = None
    self.images = images
    self.config = config
    self.shuffle = shuffle
    self.jitter = jitter
    self.norm = norm
    self.anchors = [BoundBox(0, 0, config['ANCHORS'][2*i], config['ANCHORS'][2*i+1]) for i in range(int(len(config['ANCHORS'])//2))]

    sometimes = lambda aug : iaa.Sometimes(0.5,aug)

    self.aug_pipe = iaa.Sequential(
        [
            sometimes(iaa.Affine()),
            iaa.SomeOf((0,5),
                [
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)),
                        iaa.AverageBlur(k=(2,7)),
                        iaa.MedianBlur(k=(3,11)),
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0,0.05*255),per_channel=0.5),
                    iaa.OneOf([
                        iaa.Dropout((0.01,0.1), per_channel=0.5),
                    ]),
                    iaa.Add((-10,10), per_channel=0.5),
                    iaa.Multiply((0.5,1.5), per_channel=0.5),
                    iaa.ContrastNormalization((0.5,2.0), per_channel= 0.5),
                ],
                random_order = True
            )
        ],
        random_order = True
    )
    if shuffle: np.random.shuffle(self.images)

  def __len__(self):
    return int(np.cel(float(len(self.images))/self.config['BATCH_SIZE']))

  def num_classes(self):
    return len(self.config['LABELS'])

  def size(self):
    return len(self.images)

  def load_annotation(self, i):
    annots = []

    for obj in self.images[i]['object']:
      annot = [obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax'], self.config['LABELS'].index(obj['name'])]
      annots += [annot]

    if len(annots) == 0: annots = [[]]
    return np.array(annots)

  def load_images(self, i):
    return cv2.imread(self.images[i]['filename'])

  def __getitem___(self, idx):
    l_bound = idx*self.config['BATCH_SIZE']
    r_bound = (idx+1)*self.config['BATCH_SIZE']

    if r_bound > len(self.images):
      r_bound = len(self.images)
      l_bound = r_bound - self.config['BATCH_SIZE']

    instance_count = 0

    x_batch = np.zeros((r_bound - l_bound, self.config['IMAGE_W'],3))
    b_batch = np.zeros((r_bound-l_bound, 1, 1, 1, self.config['TRUE_BOX_BUFFER'],4))
    y_batch = np.zeros((r_bound-l_bound, self.config['GRID_H'], self.config['GRID_W'], self.config['BOX'], 4+1+len(self.config['LABELS'])))

    for train_instance in self.images[l_bound:r_bound]:
      img,all_objs = self.aug_image(train_instance, jitter=self.jitter)
      true_box_index = 0

      for obj in all_objs:
        if obj['xmax'] > obj['xmin'] and obj['ymax'] > obj['ymin'] and obj['name'] in self.config['LABELS']:
              center_x = .5*(obj['xmin'] + obj['xmax'])
              center_x = center_x / (float(self.config['IMAGE_W']) / self.config['GRID_W'])
              center_y = .5*(obj['ymin'] + obj['ymax'])
              center_y = center_y / (float(self.config['IMAGE_H']) / self.config['GRID_H'])

              grid_x = int(np.floor(center_x))
              grid_y = int(np.floor(center_y))

              if grid_x < self.config['GRID_W'] and grid_y < self.config['GRID_H']:
                obj_indx = self.config['LABELS'].index(obj['name'])

                center_w = (obj['xmax']-obj['xmin'])/(float(self.config['IMAGE_W'])/ self.config['GRID_W'])













Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


IndexError: index 0 is out of bounds for axis 0 with size 0